In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
from helpers import *
from costs import *
DATA_TRAIN_PATH = '../train.csv' # TODO: download train data and supply path here 
y, x, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

Additional loss and helper functions

In [3]:
def sigmoid(t):
    """apply sigmoid function on t."""
    return np.exp(t)/(1+np.exp(t))

def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly=np.ones([x.shape[0],degree+1])
    for i in range(0,x.shape[0]):
        for j in range(1, degree+1):
            poly[i][j]=np.power(x[i,j],j)
    return poly

def compute_rmse(y,tx,w):
    e=y-(tx @ w)
    return math.sqrt(1/y.shape[0]*(e @ e))


def calculate_loss_logistic_regression(y, tx, w):
    """compute the cost by negative log likelihood."""
    a=0
    for n in range(y.shape[0]):    
        a+=np.log(1+np.exp(tx[n].T @ w))-y[n]*tx[n].T @ w
    return a[0]

def build_tx(x):
    return np.c_[np.ones((x.shape[0], 1)), x]

Gradients calculations

In [4]:
def compute_gradient_MSE(y,tx,w):
    """Compute the gradient."""
    e=y-(tx @ w)
    return -1/y.shape[0]*tx.T @ e

def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient for batch data."""
    return compute_gradient_MSE(y,tx,w)

def compute_gradient_MAE(y,tx,w):
    sumX=0
    sumY=0
    for n in range(0,y.shape[0]):
        temp=y[n]-w[0]-w[1]*tx[n,1]
        if(temp>0):
            sumX=sumX-1
            sumY=sumY-tx[n,1]
        if(temp<0):
            sumX=sumX+1
            sumY=sumY+tx[n,1]
    return np.array([sumX/y.shape[0],sumY/y.shape[0]])

def calculate_gradient_logistic_regression(y, tx, w):
    """compute the gradient of loss."""
    return tx.T @ (sigmoid(tx @ w)-y)

def calculate_hessian(y, tx, w):
    """return the hessian of the loss function."""
    # calculate hessian: 
    S=np.zeros((y.shape[0],y.shape[0]))
    for i in range(0,y.shape[0]):
        S[i,i]=sigmoid(tx[i].T @ w)*(1-sigmoid(tx[i].T @ w))
    return tx.T @ S @ tx


def logistic_regression_newton(y, tx, w):
    """return the loss, gradient, and hessian."""
    # return loss, gradient, and hessian:
    return calculate_loss_logistic_regression(y,tx,w),calculate_gradient_logistic_regression(y,tx,w),calculate_hessian(y,tx,w)

def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient, and hessian."""
    # return loss, gradient, and hessian:
    loss,gradient,hessian=logistic_regression_newton(y,tx,w)
    loss+= lambda_*np.sum(w*w)
    gradient+=lambda_*np.sum(2*w)
    hessian+=lambda_*2*w.shape[0]
    return loss,gradient,hessian

Gradient descent (one step)

In [65]:
def learning_by_gradient_descent_logistic_regression(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """
    # compute the cost:
    #loss=calculate_loss_logistic_regression(y,tx,w)
    loss=compute_loss(y,tx,w)
    # compute the gradient:
    gradient=calculate_gradient_logistic_regression(y,tx,w)
    # update w:
    w=w-gamma*gradient
    return loss, w

def learning_by_newton_method(y, tx, w, gamma):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    # return loss, gradient and hessian:
    loss,gradient,hessian = logistic_regression_newton(y,tx,w)
    # update w:
    w=w-gamma * np.linalg.inv(hessian) @ gradient
    return loss, w

def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    """
    Do one step of gradient descent, using the penalized logistic regression.
    Return the loss and updated w.
    """
    # return loss, gradient and hessian:
    
    loss,gradient,hessian = penalized_logistic_regression(y,tx,w,lambda_)
    # update w:
    w=w-gamma * np.linalg.inv(hessian) @ gradient
    return loss, w


# DO GD linear regression with mse
def learning_by_GD_mse(y,tx,w,lambda_):
    return compute_loss(y,tx,w),w-lambda_*compute_gradient_MSE(y,tx,w)
# DO SGD linear regression with mse
def learning_by_SGD_mse(y,tx,w,lambda_):
    return compute_loss(y,tx,w),w-lambda_*compute_stoch_gradient(y,tx,w)

Normal equations

In [66]:
def least_squares(y, tx):
    """calculate the least squares solution."""
    w=np.linalg.inv(tx.T @ tx) @ tx.T @ y
    return compute_loss(y,tx,w),w

def ridge_regression(y, tx, lamb):
    """implement ridge regression."""
    return np.linalg.solve((tx.T @ tx)+lamb*np.identity(tx.shape[1]), tx.T @ y)


ML Functions

In [67]:
def gradient_descent_mse(y, tx, initial_w, max_iters, gamma): 
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        loss,w=learning_by_GD_mse(y,tx,w,gamma)
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
    return losses, ws

def stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iter, gamma):
    """Stochastic gradient descent algorithm."""
    ws = [initial_w]
    losses = []
    w = initial_w
    minibatchs = batch_iter(y, tx, batch_size, num_batches=np.int(y.shape[0]/batch_size))
    num_batches=np.int(y.shape[0]/batch_size)
    for n_iter in range(0,np.int(np.min([max_iter,num_batches]))):
        # compute gradient and loss
        minibatch=minibatchs.__next__()
        loss,w=learning_by_SGD_mse(minibatch[0],minibatch[1],w,gamma)
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
    return losses, ws

def logistic_regression_gradient_descent(y, x, max_iter, threshold,gamma):
    # init parameters
    losses = []
    tx=build_tx(x)
    w = np.zeros((tx.shape[1]))
    ws=[]
    ws.append(w)
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        
        loss, w = learning_by_gradient_descent_logistic_regression(y, tx, w, gamma)
        # log info
        if iter % 1000 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criteria
        losses.append(loss)
        ws.append(w)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            print("The loss={l}".format(l=calculate_loss(y, tx, w)))
            break
    return losses,ws

def logistic_regression_newton_method(y, x,max_iters,threshold,gamma):
    # init parameters
    losses = []
    tx=build_tx(x)
    w = np.zeros((tx.shape[1]))
    ws=[]
    ws.append(w)
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_newton_method(y, tx, w, gamma)
        # log info
        if iter % 500 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criteria
        losses.append(loss)
        ws.append(w)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_newton_method")
    print("The loss={l}".format(l=calculate_loss(y, tx, w)))

def logistic_regression_penalized_gradient_descent(y, x,max_iters,threshold,gamma,lambda_):
    # init parameters
    losses = []
    tx=build_tx(x)
    w = np.zeros((tx.shape[1]))
    ws=[]
    ws.append(w)
    
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # log info
        if iter % 500 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criteria
        losses.append(loss)
        ws.append(w)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    # visualization
    visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_penalized_gradient_descent")
    print("The loss={l}".format(l=calculate_loss(y, tx, w)))
    
    
def ridge_regression_ml_function(x, y, degree,lambda_):
    """ridge regression demo."""
    # form train and test data with polynomial basis function: TODO
    # ***************************************************
    train_tx=build_poly(x,degree)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # ridge regression with different lambda: TODO
    # ***************************************************
    weight = ridge_regression(y,train_tx,lambda_)
    loss=compute_loss(y,train_tx,weight)
    return loss,weight
    
    


### Testing different methods

Gradient descent mse

In [64]:
tx=build_tx(x)
initial_w=np.ones(tx.shape[1])
max_iters=500
gamma =0.0000000001
losses,ws=gradient_descent_mse(y, tx, initial_w, max_iters, gamma)


In [74]:
losses,ws=gradient_descent_mse(y, tx, ws[len(ws)-1], max_iters, gamma)

In [77]:
final_ws=ws[len(ws)-1]

In [78]:
losses

[8415839.4050708096,
 8406843.374200752,
 8397857.6200481225,
 8388882.130832715,
 8379916.8947878303,
 8370961.9001602624,
 8362017.1352102738,
 8353082.5882115979,
 8344158.247451406,
 8335244.1012303038,
 8326340.1378623033,
 8317446.3456748277,
 8308562.7130086655,
 8299689.2282179929,
 8290825.8796703303,
 8281972.6557465335,
 8273129.5448407829,
 8264296.5353605654,
 8255473.6157266628,
 8246660.7743731244,
 8237857.9997472698,
 8229065.2803096613,
 8220282.6045340877,
 8211509.9609075692,
 8202747.3379303049,
 8193994.7241156939,
 8185252.1079902956,
 8176519.4780938393,
 8167796.8229791746,
 8159084.1312122922,
 8150381.3913722876,
 8141688.5920513505,
 8133005.7218547473,
 8124332.7694008164,
 8115669.7233209368,
 8107016.5722595369,
 8098373.3048740476,
 8089739.9098349148,
 8081116.3758255802,
 8072502.6915424475,
 8063898.8456948875,
 8055304.8270052169,
 8046720.6242086822,
 8038146.2260534475,
 8029581.6213005716,
 8021026.798724005,
 8012481.7471105745,
 8003946.45525995

In [80]:
final_ws_GD_mse=ws[len(ws)-1]

Stochastic gradient descent mse

In [130]:
tx=build_tx(x)
initial_w=np.ones(tx.shape[1])
max_iters=500
gamma =0.0000000001
losses,ws=stochastic_gradient_descent(y, tx, initial_w, x.shape[0]/500, max_iters, gamma)
losses

C:\Users\HP Pace\Documents\GitHub\MLO-project1\scripts\helpers.py:46: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


[26299049.581095859,
 25934451.944981851,
 25981873.188481964,
 25651144.554189704,
 24792840.808632888,
 24836384.749785874,
 24721839.188506033,
 25650945.29723506,
 24574953.4705474,
 24963825.578389157,
 24352119.689798065,
 25820817.11896367,
 24310115.117421348,
 26473444.457346596,
 24497785.040876105,
 25077679.806367975,
 23518083.51547448,
 24631570.806516901,
 24756384.020382691,
 24255701.042907219,
 24287826.351514623,
 23971835.341809507,
 24308094.570181288,
 23831774.240943946,
 25274601.131775174,
 23524606.605109416,
 24841835.495806899,
 24110104.154072281,
 24113587.126873393,
 25245624.085686546,
 23981919.379273389,
 23599686.497742601,
 25500834.275187507,
 25332781.25220678,
 24228077.7704252,
 24329997.020332601,
 24160098.615358658,
 24562972.20028726,
 25070706.082399383,
 24509281.364639368,
 24039549.603396341,
 24378015.51000892,
 23971672.418647032,
 21521419.463028688,
 23030965.58926126,
 25753392.156886939,
 23681802.063800052,
 24431991.89664958,
 221

In [144]:
losses,ws=stochastic_gradient_descent(y, tx, ws[len(ws)-1], x.shape[0]/500, max_iters, gamma)
losses

C:\Users\HP Pace\Documents\GitHub\MLO-project1\scripts\helpers.py:46: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


[452043.85822487425,
 448853.36532180925,
 470060.46305780654,
 445967.18903180823,
 449558.63704973413,
 400838.63414660754,
 438050.10222255631,
 404393.22570405831,
 410750.84071604331,
 410240.89389556227,
 393860.5595439162,
 433069.62857287633,
 425622.53880532511,
 458167.68581131584,
 408641.93973515561,
 386016.40313737461,
 433827.83483321487,
 459394.14743433485,
 399473.84011130035,
 447225.50659666181,
 402370.70772474422,
 468002.9539866196,
 406054.11214874225,
 427925.73597207502,
 436630.96729333256,
 379987.74917480588,
 403388.46742417646,
 483869.68217089411,
 377409.24848076818,
 454864.46648430929,
 426046.7150545519,
 448864.84372478077,
 507517.91743553808,
 456752.33096155961,
 431073.66003278148,
 400638.7846232226,
 410102.5010797276,
 468869.81406564894,
 481842.82014239958,
 400605.59786388016,
 396221.26406160719,
 425237.55916821799,
 459565.55265749351,
 446760.12730978121,
 397191.4459837458,
 413761.12345632404,
 476206.36203742289,
 412917.64004003647

In [145]:
final_SGD_gradient_ws = ws[len(ws)-1]

Least Squares

In [157]:
loss,final_LS_w=least_squares(y,tx)

In [158]:
loss

0.33944559851782757

In [159]:
final_LS_w

array([ -3.60668398e-01,   7.23061306e-05,  -7.14468236e-03,
        -6.24039951e-03,  -4.77145010e-04,  -3.08504353e-03,
         4.49444469e-04,  -2.38125455e-02,   3.42157704e-01,
        -1.09950486e-04,  -2.84186178e+00,  -2.16216038e-01,
         9.55448998e-02,   4.52028139e-02,   2.85042830e+00,
        -2.62729702e-04,  -9.95639898e-04,   2.85490239e+00,
        -3.54662999e-04,   8.49821857e-04,   3.69247838e-03,
         2.18113134e-04,  -5.00574872e-04,  -2.11522348e-01,
        -2.18882082e-04,   2.01549413e-04,   3.43135946e-04,
        -6.98404056e-05,  -6.58071046e-03,  -1.18306611e-02,
         2.84110926e+00])

Ridge Regression

In [33]:
degree=6
lambda_=100

loss,w=ridge_regression_ml_function(x,y,degree,lambda_)
loss

0.39126239486554626

In [34]:
test_loss=compute_loss(_,build_poly(tX_test,degree),w)

In [35]:
test_loss

0.30849080908943788

Logistic regression

In [80]:
max_iters=500
threshold=0.0000001
gamma=0.000000000000001


loss,ws=logistic_regression_gradient_descent(y,x,max_iters,threshold,gamma)

Current iteration=0, the loss=0.5


In [81]:
loss

[0.5,
 0.49961914534872515,
 0.49923960974446357,
 0.49886139186659062,
 0.49848449039544634,
 0.49810890401233782,
 0.49773463139953955,
 0.49736167124029468,
 0.49699002221881711,
 0.49661968302029197,
 0.49625065233087701,
 0.49588292883770502,
 0.49551651122888296,
 0.49515139819349513,
 0.49478758842160331,
 0.4944250806042485,
 0.49406387343345254,
 0.49370396560221813,
 0.49334535580453076,
 0.49298804273536084,
 0.49263202509066256,
 0.49227730156737798,
 0.49192387086343536,
 0.4915717316777532,
 0.49122088271023873,
 0.49087132266179118,
 0.49052305023430159,
 0.49017606413065445,
 0.48983036305472943,
 0.48948594571140114,
 0.48914281080654182,
 0.48880095704702137,
 0.48846038314070911,
 0.48812108779647434,
 0.48778306972418778,
 0.48744632763472323,
 0.4871108602399572,
 0.48677666625277161,
 0.48644374438705379,
 0.48611209335769795,
 0.48578171188060654,
 0.48545259867269036,
 0.48512475245187103,
 0.48479817193708075,
 0.48447285584826388,
 0.48414880290637841,
 0.4838

In [87]:

test_loss=compute_loss(_,build_tx(tX_test),w[len(w)-1])

ValueError: operands could not be broadcast together with shapes (500,) (568238,) 

Newton method

## Generate predictions and save ouput in csv format for submission:

In [18]:
DATA_TEST_PATH = '../test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [38]:
print(_.shape)
print(tX_test.shape)
print(ids_test.shape)


loss=compute_loss(_,build_tx(tX_test),final_LS_w)

loss

()
(568238, 30)
(568238,)


NameError: name 'final_LS_w' is not defined

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)